In [1]:
setwd("/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/bulk/hahn_2023/cortex")

source("/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/code/enrichment_fxns.R")

source("/mnt/lareaulab/reliscu/code/upper_first.R")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last




Here I perform enrichment analysis to find modules enriched for cell type markers. 

These modules will later be used to correlate with exon PSI to define cell type-specific exons.

In [2]:
mod_def <- "PosBC"

# Prep marker genes

## Yao et. al 2021 marker genes

In [3]:
# # Use marker genes from Yao et. al 2021:

# marker_genes_df <- fread("/mnt/lareaulab/reliscu/projects/NSF_GRFP/data/scRNA-seq/yao_2021/yao_2021_tableS6_marker_genes.txt", data.table=FALSE)

# head(marker_genes_df)

In [4]:
# ctypes <- unique(marker_genes_df$cl1_cluster_label)

# marker_genes_list <- lapply(ctypes, function(w_ctype) {
#     working_ctype_tests <- marker_genes_df[(marker_genes_df$cl1_cluster_label %in% w_ctype),]
#     de_genes_per_test <- lapply(ctypes[ctypes != w_ctype], function(o_ctype) {
#         if (o_ctype != w_ctype) {
#             working_ctype_tests$gene[(working_ctype_tests$cl2_cluster_label %in% o_ctype) & (working_ctype_tests$direction %in% "up")]
#         }
#     })
#     hits_per_gene <- table(unlist(de_genes_per_test))
#     names(hits_per_gene)[hits_per_gene > 60] 
# })
# names(marker_genes_list) <- ctypes

In [5]:
# marker_genes_list <- marker_genes_list[lengths(marker_genes_list) > 1]

## Pseudobulk DE genes

In [6]:
unique <- FALSE

In [7]:
# pairwise_res_list <- readRDS("/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_pairwise_DE_genes_dream.RDS")
# pairwise_ctype_genes <- prep_DE_genes(pairwise_res_list, lfc_threshold, pairwise=TRUE, unique=unique)

pairwise_res_list <- readRDS("/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/yao_2021/ACA/data/DE_genes/yao_2021_ACA_STAR_donor_cell_type_pseudobulk_pairwise_DE_genes_dream.RDS")
pairwise_ctype_genes <- prep_DE_genes(pairwise_res_list, lfc_threshold, pairwise=TRUE, unique=unique)

marker_genes_list <- pairwise_ctype_genes

## MO marker genes

In [8]:
load("/mnt/lareaulab/reliscu/data/gene_sets/Oldham/MO_sets.RData")

In [9]:
# MO_legend[grepl("GLUTA", MO_legend$SetName) & grepl("CellType", MO_legend$Category),]

In [10]:
sets <- c(
    "BARRES_ASTROCYTES",
    "BARRES_OLIGODENDROCYTES",
    "HICKMAN_MICROGLIA_SENSOME_99",
    "BARRES_NEURONS",
    "BUTLER_ENDOTHELIAL_HPA",
    "ZEISEL_MURAL",
    "ZEISEL_EPENDYMAL",
    "ZEISEL_INTERNEURON",
    "ZHANG_MOUSE_OPC_2014",
    "Mukamel_InhibitoryNeurons",
    "Mukamel_ExcitatoryNeurons",
    "SUGINO_UP_GABAERGIC_NEURONS",
    "SUGINO_UP_GLUTAMATERGIC_NEURONS",
    "SUGINO_UP_LAYERS4-6_GABAERGIC_NEURONS_CINGULATE_CTX",
    "BAKKEN_2019_PVALB_GABAERGIC_DE_GABA_CLUSTERS",
    "BAKKEN_2019_VIP_GABAERGIC_DE_GABA_CLUSTERS",
    "BAKKEN_2019_SST_GABAERGIC_DE_GABA_CLUSTERS",
    "BAKKEN_2019_LAMP5_GABAERGIC_DE_GABA_CLUSTERS",
    "BAKKEN_2019_SNCG_GABAERGIC_DE_GABA_CLUSTERS",
    "BAKKEN_2019_SST_CHODL_GABAERGIC_DE_GABA_CLUSTERS",
    "VELMESHEV_2019_IN_SST",
    MO_legend$SetName[grepl("schirmer", MO_legend$SetName, ignore.case=T)],
    MO_legend$SetName[grepl("yang_pfc", MO_legend$SetName, ignore.case=T)]
)

In [11]:
mask <- MO_legend$SetName %in% sets

marker_genes_list <- MO_sets[mask]
names(marker_genes_list) <- MO_legend$SetName[mask] 

# Format genes to match mouse symbols

marker_genes_list <- lapply(marker_genes_list, function(x) unname(sapply(x, upper_first)))

# Get enrichments

In [12]:
set_source <- "MO"

In [13]:
network_dir <- "hahn_2023_cortex_STAR_TPM_OR_mergeParam0.85_Modules"

In [14]:
enrichments_df <- get_module_enrichments(network_dir, marker_genes_list, mod_def)

In [15]:
# Get most enriched cell type for each module
# If cell type is most enriched in multiple modules, choose module with smallest p-value

top_mods_df <- enrichments_df %>%
    group_by(Cell_type) %>%
    slice_min(Qval, with_ties=FALSE) %>%
    group_by(Network, Module) %>%
    arrange(Network) %>%
    slice_min(Qval, with_ties=FALSE) %>%
    arrange(Qval)

In [10]:
top_mods_df[,c("Cell_type", "Pval", "Qval", "Module", "Network")]

Cell_type,Pval,Qval,Module,Network
<chr>,<dbl>,<dbl>,<chr>,<chr>
YANG_PFC_2021_OLIGODENDROCYTE,9.480520e-103,7.525921e-98,brown,Bicor-None_signum0.379_minSize12_merge_ME_0.85_20371
BARRES_OLIGODENDROCYTES,1.032214e-83,2.926439e-80,brown,Bicor-None_signum0.379_minSize10_merge_ME_0.85_20371
YANG_PFC_2021_NRGN_NEURON,3.647019e-48,4.746087e-45,turquoise,Bicor-None_signum0.476_minSize10_merge_ME_0.85_20371
BAKKEN_2019_PVALB_GABAERGIC_DE_GABA_CLUSTERS,3.746042e-45,4.505638e-42,turquoise,Bicor-None_signum0.476_minSize6_merge_ME_0.85_20371
YANG_PFC_2021_L2_3_EXCITATORY_NEURON,8.225316e-41,8.161878e-38,turquoise,Bicor-None_signum0.476_minSize8_merge_ME_0.85_20371
BAKKEN_2019_SST_CHODL_GABAERGIC_DE_GABA_CLUSTERS,7.550746e-39,6.660010e-36,black,Bicor-None_signum0.627_minSize8_merge_ME_0.85_20371
YANG_PFC_2021_ASTROCYTE,4.708731e-35,2.943253e-32,green,Bicor-None_signum0.627_minSize8_merge_ME_0.85_20371
Mukamel_ExcitatoryNeurons,8.260739e-31,4.098514e-28,turquoise,Bicor-None_signum0.379_minSize12_merge_ME_0.85_20371
YANG_PFC_2021_OPC,3.287553e-25,1.197137e-22,blue,Bicor-None_signum0.379_minSize8_merge_ME_0.85_20371


In [13]:
write.csv(top_mods_df, file=paste0("data/enrichments/hahn_2023_cortex_STAR_TPM_OR_QN_mergeParam0.85_top_", set_source, "_Qval_modules.csv"), row.names=FALSE, quote=FALSE)